In [ ]:
!pip install geopandas contextily

In [2]:
import geopandas as gdp 
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx

In [5]:
quartieri = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/ds964_nil_wm.zip")
bici = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/bike_ciclabili.zip")
regioni = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/Reg01012022_g (1)")
province = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/ProvCM01012022_g")
fontane = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/Fontanelle.zip")
fiumi = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/Corsi_acqua_Piano_di_Gestione.zip")

In [11]:
comuni = gdp.read_file("/content/drive/MyDrive/Colab Notebooks/Com01012022_g")

1. visualizzare i primi dieci record del file dei quartieri

In [ ]:
quartieri.head(10)

2. visualizzare presente se la regione Alsazia è presente, altrimenti mostrare non presente se la regione non c'è

In [7]:
if len(regioni[regioni["DEN_REG"] == "Alsazia"]) > 0:
  print("presente")
else:
  print("non presente")

non presente


3. visualizzare su una mappa di milano i quartieri e le piste ciclabili

In [8]:
quartieri3857 = quartieri.to_crs(3857)
bici3857 = bici.to_crs(3857)

In [ ]:
ax = quartieri3857.plot(figsize = (15, 12), edgecolor = "k", facecolor = "none")
bici3857.plot(ax = ax, edgecolor = "red", linewidth = 2)
ctx.add_basemap(ax)

4. dire se il comune inserito dall'utente è in lombardia. In caso sia in lombardia visualizzarlo, se invece non è in lombardia mostrare un messaggio opportuno

In [12]:
regioni3857 = regioni.to_crs(3857)
comuni3857 = comuni.to_crs(3857)

In [13]:
lombardia = regioni3857[regioni3857["DEN_REG"] == "Lombardia"].geometry.item()

In [15]:
comune_inserito = comuni3857[comuni3857["COMUNE"] == input("inserire un comune: ")].geometry.item()
if comune_inserito.intersects(lombardia) == True:
  comune_inseritoSerie = gdp.GeoSeries([comune_inserito])
  ax = comune_inseritoSerie.plot(figsize = (15, 12), edgecolor = "k", facecolor = "none")
  ctx.add_basemap(ax)
else:
  print("comune non in lombardia")

inserire un comune: Roma
comune non in lombardia


5. visualizzare colori diversi sui quartieri di milano in base al numero di fontanelle

In [16]:
fontane3857 = fontane.to_crs(3857)

In [ ]:
quartieri3857

In [ ]:
fontane3857

In [ ]:
joined = gdp.sjoin(fontane3857, quartieri3857, predicate="within", how="left")
joined

In [ ]:
fontane_quartiere = joined.groupby("NIL").count()[["Licenza"]].reset_index()
fontane_quartiere

In [ ]:
finale = quartieri3857.merge(fontane_quartiere, on="NIL")
finale

In [ ]:
ax = finale.plot(figsize = (15, 12), edgecolor = "k", column = "Licenza", legend = True)
ctx.add_basemap(ax)